In [6]:
import adapters
from transformers import AutoModel, AutoTokenizer
from adapters import PrefixTuningConfig


# Load pre-trained model and tokenizer
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
adapters.init(model) # prepare model for use with adapters


config = PrefixTuningConfig(flat=False, prefix_length=30)
model.add_adapter("prefix_tuning", config=config)


model.delele_all_adapters()

print(model.active_adapters)


In [3]:
model.save_adapter("ngoc", "prefix_tuning")

In [8]:
import torch
from transformers import BertModel, BertTokenizer
import numpy as np

# Khởi tạo mô hình và tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Dữ liệu đầu vào
sentences = ["Hello, how are you?", "I am fine, thank you!"]

# Tokenize dữ liệu
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)

# Forward dữ liệu qua mô hình
with torch.no_grad():
    outputs = model(**inputs)

# Lấy embedding của token `[CLS]`
cls_embeddings = outputs.last_hidden_state[:, 0, :]

# Tính mean của embedding
mean_embedding = torch.mean(cls_embeddings, dim=0)

# Tính covariance của embedding
covariance = torch.matmul(cls_embeddings.T, cls_embeddings) / cls_embeddings.shape[0]

# Chuyển tensor sang numpy array để hiển thị
mean_embedding_np = mean_embedding.numpy()
covariance_np = covariance.numpy()

print("Mean embedding:", mean_embedding_np.shape)
print("Covariance matrix:", covariance_np.shape)


Mean embedding: (768,)
Covariance matrix: (768, 768)
